In [3]:
!pip install -q vllm fastapi pyngrok uvicorn

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "/home/yuan/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/home/yuan/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/yuan/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "/tmp/pip-build-env-m3q2zwb3/overlay/lib/python3.11/site-packages/setuptools/build_meta.py", line 355, in get_requires_for_buil

In [ ]:
'''
curl 'https://17ca-34-32-204-221.ngrok.io' \
  -d '{
  "prompt": "is god exist?",
  "tokens": 512,
  "temp": 0.7,
  "top_p": 0.95,
  "repetition_penalty": 1.15
}' \
  -H 'Content-Type: application/json'
'''

'''
import requests

url = 'https://e538-34-32-204-221.ngrok.io'
data = {
    "prompt": "is god exist?",
    "tokens": 512,
    "temp": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.15
}
headers = {'Content-Type': 'application/json'}

response = requests.post(url, json=data, headers=headers)

print(response.text)

'''

In [4]:
from vllm import LLM, SamplingParams

ModuleNotFoundError: No module named 'vllm'

In [ ]:
class models:
    def __init__(self):
        self.model_a = None
        self.model_b = None
        self.model_a_name = None
        self.model_b_name = None
    
    def load_models(self, model_names: list):
        if self.model_a_name == model_names[0] and self.model_b_name == model_names[1]:
            return
        elif ((self.model_a_name == model_names[0] and self.model_b_name != model_names[1])
              or (self.model_a_name != model_names[0] and self.model_b_name == model_names[1])):
            if self.model_a_name == model_names[0]:
                self.model_b = LLM(model_names[1])
                self.model_b_name = model_names[1]
            else:
                self.model_a = LLM(model_names[1])
                self.model_a_name = model_names[1]
        else:
            self.model_a = LLM(model_names[0])
            self.model_a_name = model_names[0]
            self.model_b = LLM(model_names[1])
            self.model_b_name = model_names[1]
    
    def generate(self, model_name, prompt, temperature, top_p):
        if model_name == self.model_a_name:
            model = self.model_a
        else:
            model = self.model_b
            
        sampling_params = SamplingParams(
            temperature=temperature,
            top_p=top_p,
        )
        
        return model.generate(prompt, sampling_params=sampling_params)

In [ ]:
from fastapi import FastAPI, Form
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

app = FastAPI()
model_class = models()

class GenerateRequest(BaseModel):
    prompt: str
    model_name: list = current_model_names
    temp: float = 0.7
    top_p: float = 0.95
    
class ModelRequest(BaseModel):
    model_names: list
    
@app.get('/')
async def root():
    rick_roll_code = '<iframe width="560" height="315" src="https://www.youtube.com/embed/dQw4w9WgXcQ?autoplay=1&mute=1" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
    return rick_roll_code

@app.post('/generate')
async def generate(request: GenerateRequest):
    model_class.load_models(request.model_name)
    return model_class.generate(request.model_name[0], request.prompt, request.temp, request.top_p)

@app.post('/model')
async def load_model(request: ModelRequest):
    if type(request.model_names) == str:
        return {'error': 'model name is not a list'}
    else:
        model_class.load_models(request.model_names)
        return {'current_model_name': request.model_names}


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)
